In [67]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import pymongo

In [68]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.recipe_db
collection = db.low_calorie
url = 'https://www.allrecipes.com/recipes/84/healthy-recipes/'

response = requests.get(url)

soup = bs(response.text, 'html.parser')

In [69]:
results = soup.find_all('a', class_='carouselNav__link recipeCarousel__link')
results

[<a class="carouselNav__link recipeCarousel__link" data-tracking-content-headline="healthy appetizers" href="https://www.allrecipes.com/recipes/1319/healthy-recipes/appetizers/" tabindex="0">
 <div alt="Healthy Appetizers" aria-label="" class="component lazy-image lazy-image-udf aspect_1x1 align-default" data-alt="Healthy Appetizers" data-content-type="" data-crop-percentage="100" data-high-density="false" data-main-recipe="" data-orientation="default" data-original-height="1277" data-original-width="1278" data-shop-image="false" data-special-crop="" data-src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2F2020%2F07%2F22%2F6583475.jpg" data-title="Healthy Appetizers" data-tracking-zone="image" data-width="      100
                                    ">
 <noscript>
 <img alt="Healthy Appetizers" src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2

In [70]:
health_cats = ['low-calorie recipes','low-carb recipes','low-cholesterol recipes','low-fat recipes','weight-loss recipes','healthy snacks']

In [71]:
cat =  [x['href'] for x in results if x['data-tracking-content-headline'] in health_cats]
cat

['https://www.allrecipes.com/recipes/12155/healthy-recipes/snacks/',
 'https://www.allrecipes.com/recipes/1232/healthy-recipes/low-calorie/',
 'https://www.allrecipes.com/recipes/742/healthy-recipes/low-carb/',
 'https://www.allrecipes.com/recipes/737/healthy-recipes/low-cholesterol/',
 'https://www.allrecipes.com/recipes/1231/healthy-recipes/low-fat/',
 'https://www.allrecipes.com/recipes/22607/healthy-recipes/weight-loss/']

In [72]:
main_cats={}

for c in cat:
    r = requests.get(c)
    soup = bs(r.text, 'html.parser')
    title = soup.find('h1', class_= 'category-page-heading').text
    results = soup.find_all('a', class_= 'card__titleLink manual-link-behavior')
    links = [y['href'] for y in results]
    
    main_cats[title] = list(set(links))
    
main_cats

{'Healthy Snack Recipes': ['https://www.allrecipes.com/recipe/255799/black-bean-hummus-without-tahini/',
  'https://www.allrecipes.com/recipe/261342/baked-banana-chips/',
  'https://www.allrecipes.com/recipe/15006/spiced-sweet-roasted-red-pepper-hummus/',
  'https://www.allrecipes.com/recipe/239017/fresh-tomato-salsa-restaurant-style/',
  'https://www.allrecipes.com/recipe/223112/first-baby-food-peas/',
  'https://www.allrecipes.com/recipe/21108/oatmeal-whole-wheat-quick-bread/',
  'https://www.allrecipes.com/recipe/246372/perfectly-dry-roasted-chickpeas/',
  'https://www.allrecipes.com/recipe/240551/fruit-skewers-with-apple-cinnamon-dipping-sauce/',
  'https://www.allrecipes.com/recipe/14860/roasted-garlic/',
  'https://www.allrecipes.com/recipe/214722/pickled-daikon-radish-and-carrot/',
  'https://www.allrecipes.com/recipe/24720/texas-caviar-i/',
  'https://www.allrecipes.com/recipe/16259/ds-famous-salsa/',
  'https://www.allrecipes.com/recipe/219456/healthier-buffalo-chicken-dip/',


In [73]:
for k, v in main_cats.items():
    collection = db[k]
    for u in v:
        print(u)
        resp = requests.get(u)
        soup = bs(resp.text, 'html.parser')
        title = soup.find('h1', class_= 'headline heading-content').text
        
        try:
            rating = soup.find('span', class_='review-star-text').text.replace('\n',"").strip()
        except:
            rating = 'none'
        
        try:
            num_ratings = soup.find('span', class_='ugc-ratings-item').text.replace('\n',"").strip()
        except:
            num_ratings = 'none'
            
        
        time = soup.find_all('div', class_= 'recipe-meta-item')
        time_dict = {}
        time_dict['prep'] = time[0].find('div', class_= 'recipe-meta-item-body').text.replace('\n',"").strip()
        time_dict['total'] = time[1].find('div', class_= 'recipe-meta-item-body').text.replace('\n',"").strip()
        ingredients = soup.find_all('span', class_='ingredients-item-name')
        ingredient_list = [ingredient.text.replace('\n',"").strip() for ingredient in ingredients]
        nutrients = soup.find_all('span', class_='nutrient-name')
        nutrient_dict = {}
        for nutrient in nutrients:
            name = nutrient.text.replace('\n',"").strip().split(':')
            nutrient_dict[name[0]] = name[1].strip()
        image = soup.find('div', class_='image-container').noscript.img['src']
        
        print('---------------------')
        print(k)
        print(title)
        print(rating)
        print(num_ratings)
        print(time_dict)
        print(ingredient_list)
        print(nutrient_dict)
        print(collection)
        
        post = {
            'title': title,
            'rating': rating,
            'number_of_ratings': num_ratings,
            'times': time_dict,
            'ingredients': ingredient_list,
            'nutrients': nutrient_dict,
            'image_url': image
        }
         
        collection.insert_one(post)

https://www.allrecipes.com/recipe/255799/black-bean-hummus-without-tahini/
---------------------
Healthy Snack Recipes
Black Bean Hummus without Tahini
Rating: 5 stars
3 Ratings
{'prep': '10 mins', 'total': '10 mins'}
['1 (15 ounce) can no-salt-added black beans, drained and rinsed', '¼ cup fresh cilantro', '¼ cup lime juice', '1  jalapeno pepper, trimmed and seeded', '1 tablespoon sesame oil', '4 cloves garlic', '1 teaspoon ground cumin', '¼ teaspoon ground paprika', '¼ teaspoon cayenne pepper']
{'protein': '3.1g', 'carbohydrates': '8.8g', 'dietary fiber': '2.6g', 'sugars': '0.2g', 'fat': '1.8g', 'saturated fat': '0.3g', 'vitamin a iu': '165.3IU', 'niacin equivalents': '0.1mg', 'vitamin c': '3.3mg', 'folate': '1.9mcg', 'calcium': '7.4mg', 'iron': '0.3mg', 'magnesium': '2.5mg', 'potassium': '144.7mg', 'sodium': '7.7mg', 'calories from fat': '16.2'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Healthy Snack 

IndexError: list index out of range

In [74]:
df = pd.read_csv("/Users/jacobzacarias/Downloads/Recipes.csv")
print(df)

                         _id                                   title  \
0   6035497bb13fad309cccb570        Black Bean Hummus without Tahini   
1   6035497bb13fad309cccb571                      Baked Banana Chips   
2   6035497fb13fad309cccb572  Spiced Sweet Roasted Red Pepper Hummus   
3   60354981b13fad309cccb573     Fresh Tomato Salsa Restaurant-Style   
4   60354982b13fad309cccb574                   First Baby Food: Peas   
..                       ...                                     ...   
76  60354b12b13fad309cccb5bd           Salt and Pepper Skillet Fries   
77  60354b12b13fad309cccb5be                    Honey Wheat Bread II   
78  60354b13b13fad309cccb5bf                   Roti Bread from India   
79  60354b13b13fad309cccb5c0                           Baked Bananas   
80  60354b13b13fad309cccb5c1                            Granola Bars   

                rating number_of_ratings times.prep times.total  \
0      Rating: 5 stars         3 Ratings    10 mins     10 mins   
1

In [75]:
#4 last digits in ID
df['_id'] = df['_id'].str[-4:]
print(df['_id'])

0     b570
1     b571
2     b572
3     b573
4     b574
      ... 
76    b5bd
77    b5be
78    b5bf
79    b5c0
80    b5c1
Name: _id, Length: 81, dtype: object


In [76]:
df['rating'] = df['rating'].str.replace('Rating:','')
df['rating'] = df['rating'].str.replace('stars','')
#over 50 reviews 
df['number_of_ratings'] = df['number_of_ratings'].str.replace('Ratings','')
df['number_of_ratings'] = df['number_of_ratings'].str.replace(',','')
df['number_of_ratings']= df['number_of_ratings'].astype(str).astype(int)
df = df[df['number_of_ratings'] >= 50]
#ratings over 4.0 
df['rating'] = df['rating'].str.replace('.','')
df['rating']= df['rating'].astype(str).astype(int)
df = df[df['rating'] >= 400]
df['rating'] = df['rating'].astype(int)
df['rating'] = df['rating'].apply(lambda x: x / 10 ** (len((str(x))) - 1))
(df)

,_id,title,rating,number_of_ratings,times.prep,times.total,ingredients,nutrients.protein,nutrients.carbohydrates,nutrients.dietary fiber,...,nutrients.niacin equivalents,nutrients.vitamin c,nutrients.folate,nutrients.calcium,nutrients.iron,nutrients.magnesium,nutrients.potassium,nutrients.sodium,nutrients.calories from fat,image_url
2,b572,Spiced Sweet Roasted Red Pepper Hummus,4.67,725,15 mins,1 hr,"[""1 (15 ounce) can garbanzo beans, drained"",""1...",2.5g,9.6g,2.1g,...,0.6mg,11.3mg,27.3mcg,23.2mg,0.8mg,13mg,106.6mg,370.3mg,19.4,https://imagesvc.meredithcorp.io/v3/mm/image?u...
5,b575,Oatmeal Whole Wheat Quick Bread,4.18,355,20 mins,20 mins,"[""1 cup rolled oats"",""1 cup whole wheat flour""...",2.9g,15.1g,1.9g,...,1.4mg,0.1mg,7.6mcg,64.1mg,0.8mg,25.6mg,96.9mg,166.2mg,19.5,https://imagesvc.meredithcorp.io/v3/mm/image?u...
8,b578,Roasted Garlic,4.53,148,5 mins,1 hr,"[""10 medium heads garlic"",""3 tablespoons olive...",2.4g,12.3g,0.8g,...,0.7mg,11.6mg,1.1mcg,67.6mg,0.6mg,9.3mg,149.7mg,6.4mg,26.0,https://imagesvc.meredithcorp.io/v3/mm/image?u...
10,b57a,Texas Caviar I,4.83,344,15 mins,1 hr,"[""½ onion, chopped"",""1 green bell pepper, ch...",3.5g,11.8g,3.7g,...,1.2mg,15.8mg,45.8mcg,31.5mg,1.2mg,24.1mg,254mg,414.7mg,48.5,https://imagesvc.meredithcorp.io/v3/mm/image?u...
11,b57b,D's Famous Salsa,4.66,1169,10 mins,10 mins,"[""2 (14.5 ounce) cans stewed tomatoes"",""½ oni...",0.6g,3.9g,0.7g,...,0.5mg,6.5mg,3.9mcg,19.7mg,0.7mg,7mg,119.7mg,282.9mg,1.0,https://imagesvc.meredithcorp.io/v3/mm/image?u...
12,b57c,Healthier Buffalo Chicken Dip,4.85,78,5 mins,40 mins,"[""2 (10 ounce) cans natural chunk chicken, dra...",11g,10.7g,0.4g,...,3.4mg,7.7mg,13.9mcg,32.3mg,0.6mg,8.6mg,147.1mg,784.1mg,143.5,https://imagesvc.meredithcorp.io/v3/mm/image?u...
13,b57d,Balsamic Bruschetta,4.75,1133,15 mins,15 mins,"[""8 roma (plum) tomatoes, diced"",""⅓ cup chopp...",8.3g,35.2g,2.2g,...,4.5mg,8.6mg,94.8mcg,64.2mg,2.4mg,25.1mg,233.2mg,470.9mg,22.4,https://imagesvc.meredithcorp.io/v3/mm/image?u...
14,b57e,Baked Kale Chips,4.48,2597,10 mins,10 mins,"[""1 bunch kale"",""1 tablespoon olive oil"",""1 te...",2.5g,7.6g,1.5g,...,1.3mg,89.7mg,21.7mcg,104mg,1.3mg,25.7mg,337.2mg,185.1mg,25.1,https://imagesvc.meredithcorp.io/v3/mm/image?u...
15,b57f,Honey Wheat Bread III,4.34,74,30 mins,40 mins,"[""¾ cup warm water"",""1 (.25 ounce) package act...",4.9g,30.6g,2.6g,...,3.6mg,NaN,56.8mcg,23.4mg,1.7mg,28.5mg,116.2mg,161.6mg,23.4,https://imagesvc.meredithcorp.io/v3/mm/image?u...
17,b581,Salsa,4.31,250,10 mins,10 mins,"[""4 large tomatoes, chopped"",""1 onion, choppe...",2.3g,11.7g,3.1g,...,1.7mg,31.1mg,38.8mcg,33.9mg,0.8mg,27.5mg,545.3mg,13.4mg,4.9,https://imagesvc.meredithcorp.io/v3/mm/image?u...


In [81]:
df.columns = ['ID', 'Title','Rating','Ratings_Received','Time_Prepared',
              'Total_Time','Ingredients','Protein','Carbohydrates','Fiber','','','','',
              'Niacin Equivalents','Vitamin C','Folate','Calcium','Iron','Magnesium',
              'Potassium','Sodium','Calories from Fat','Image_Url']
df

,ID,Title,Rating,Ratings_Received,Time_Prepared,Total_Time,Ingredients,Protein,Carbohydrates,Fiber,...,Niacin Equivalents,Vitamin C,Folate,Calcium,Iron,Magnesium,Potassium,Sodium,Calories from Fat,Image_Url
2,b572,Spiced Sweet Roasted Red Pepper Hummus,4.67,725,15 mins,1 hr,"[""1 (15 ounce) can garbanzo beans, drained"",""1...",2.5g,9.6g,2.1g,...,0.6mg,11.3mg,27.3mcg,23.2mg,0.8mg,13mg,106.6mg,370.3mg,19.4,https://imagesvc.meredithcorp.io/v3/mm/image?u...
5,b575,Oatmeal Whole Wheat Quick Bread,4.18,355,20 mins,20 mins,"[""1 cup rolled oats"",""1 cup whole wheat flour""...",2.9g,15.1g,1.9g,...,1.4mg,0.1mg,7.6mcg,64.1mg,0.8mg,25.6mg,96.9mg,166.2mg,19.5,https://imagesvc.meredithcorp.io/v3/mm/image?u...
8,b578,Roasted Garlic,4.53,148,5 mins,1 hr,"[""10 medium heads garlic"",""3 tablespoons olive...",2.4g,12.3g,0.8g,...,0.7mg,11.6mg,1.1mcg,67.6mg,0.6mg,9.3mg,149.7mg,6.4mg,26.0,https://imagesvc.meredithcorp.io/v3/mm/image?u...
10,b57a,Texas Caviar I,4.83,344,15 mins,1 hr,"[""½ onion, chopped"",""1 green bell pepper, ch...",3.5g,11.8g,3.7g,...,1.2mg,15.8mg,45.8mcg,31.5mg,1.2mg,24.1mg,254mg,414.7mg,48.5,https://imagesvc.meredithcorp.io/v3/mm/image?u...
11,b57b,D's Famous Salsa,4.66,1169,10 mins,10 mins,"[""2 (14.5 ounce) cans stewed tomatoes"",""½ oni...",0.6g,3.9g,0.7g,...,0.5mg,6.5mg,3.9mcg,19.7mg,0.7mg,7mg,119.7mg,282.9mg,1.0,https://imagesvc.meredithcorp.io/v3/mm/image?u...
12,b57c,Healthier Buffalo Chicken Dip,4.85,78,5 mins,40 mins,"[""2 (10 ounce) cans natural chunk chicken, dra...",11g,10.7g,0.4g,...,3.4mg,7.7mg,13.9mcg,32.3mg,0.6mg,8.6mg,147.1mg,784.1mg,143.5,https://imagesvc.meredithcorp.io/v3/mm/image?u...
13,b57d,Balsamic Bruschetta,4.75,1133,15 mins,15 mins,"[""8 roma (plum) tomatoes, diced"",""⅓ cup chopp...",8.3g,35.2g,2.2g,...,4.5mg,8.6mg,94.8mcg,64.2mg,2.4mg,25.1mg,233.2mg,470.9mg,22.4,https://imagesvc.meredithcorp.io/v3/mm/image?u...
14,b57e,Baked Kale Chips,4.48,2597,10 mins,10 mins,"[""1 bunch kale"",""1 tablespoon olive oil"",""1 te...",2.5g,7.6g,1.5g,...,1.3mg,89.7mg,21.7mcg,104mg,1.3mg,25.7mg,337.2mg,185.1mg,25.1,https://imagesvc.meredithcorp.io/v3/mm/image?u...
15,b57f,Honey Wheat Bread III,4.34,74,30 mins,40 mins,"[""¾ cup warm water"",""1 (.25 ounce) package act...",4.9g,30.6g,2.6g,...,3.6mg,NaN,56.8mcg,23.4mg,1.7mg,28.5mg,116.2mg,161.6mg,23.4,https://imagesvc.meredithcorp.io/v3/mm/image?u...
17,b581,Salsa,4.31,250,10 mins,10 mins,"[""4 large tomatoes, chopped"",""1 onion, choppe...",2.3g,11.7g,3.1g,...,1.7mg,31.1mg,38.8mcg,33.9mg,0.8mg,27.5mg,545.3mg,13.4mg,4.9,https://imagesvc.meredithcorp.io/v3/mm/image?u...
